**Import Libraries**

In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='darkgrid')
pd.set_option('display.float_format',lambda  x: '%.2f' %x)


**Load Data**

In [ ]:
sales = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_cats = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')

In [ ]:
print('sales:',sales.shape,'test:',test.shape,'items:',items.shape,'item_cats:',item_cats.shape,'shop:',shops.shape)

Change date form to DDMMYYYY

In [ ]:
sales['date'] = pd.to_datetime(sales['date'], format='%d.%m.%Y')

In [ ]:
sales.head(3)


In [ ]:
test.head(3)

In [ ]:
test.shape


In [ ]:
items.head(3)

In [ ]:
item_cats.head(3)

In [ ]:
shops.head(3)

In [ ]:
sales[sales['item_price']<=0]

In [ ]:
sales[(sales.shop_id==32)&(sales.item_id==2973)&(sales.date_block_num==4)]

In [ ]:
median = sales[(sales.shop_id==32)&(sales.item_id==2973)&(sales.date_block_num==4)&
               (sales.item_price>0)].item_price.median()

In [ ]:
sales.loc[sales.item_price<0,'item_price'] =median

**Pivot Table **

we want get total count value of an item over the whole month for a shop. 
That why we made shop_id and item_id our indices and date_block_num our column 
the value we want is item_cnt_day and used sum as aggregating function 

In [ ]:

dataset = sales.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum')

In [ ]:
dataset

Reset our indices, so that data should be in way we can easily manipulate

In [ ]:

dataset.reset_index(inplace = True)

In [ ]:
dataset.head()

In [ ]:
dataset.shape

Merge our pivot table with the test_data because we want to keep the data for prediction

In [ ]:
dataset = pd.merge(test,dataset,on = ['item_id','shop_id'],how = 'left')

In [ ]:
dataset

Lets fill all NaN values with 0

In [ ]:
dataset.fillna(0,inplace = True)

dataset.head()

Drop shop_id and item_id because we do not need them

In [ ]:
dataset.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)
dataset.head()

In [ ]:
# X we will keep all columns execpt the last one 
X_train = np.expand_dims(dataset.values[:,:-1],axis = 2)
# the last column is our label
y_train = dataset.values[:,-1:]

# for test we keep all the columns execpt the first one
X_test = np.expand_dims(dataset.values[:,1:],axis = 2)

# lets have a look on the shape 
print(X_train.shape,y_train.shape,X_test.shape)

In [ ]:
# importing libraries 
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout

In [ ]:
my_model = Sequential()
my_model.add(LSTM(units = 64,input_shape = (33,1)))
my_model.add(Dropout(0.3)) #The dropout rate is set to 30%, meaning one in 3.33 inputs will be randomly excluded from each update cycle.
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()

In [ ]:
lst_pred = my_model.fit(X_train,y_train,batch_size = 4000,epochs = 8)

In [ ]:
y_pred = my_model.predict(X_test)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

### Grid search

For simplicity we are keeping less parameters.

In [ ]:

def build_classifier(optimizer):
    grid_model = Sequential()
    grid_model.add(LSTM(units = 64,input_shape = (33,1)))
    grid_model.add(Dropout(0.4))
    grid_model.add(Dense(1))

    grid_model.compile(loss = 'mse',optimizer = optimizer, metrics = ['mean_squared_error'])
    return my_model

grid_model = KerasClassifier(build_fn=build_classifier)
parameters = {'batch_size' : [4000,4080],
              'epochs' : [8,10],
              'optimizer' : ['adam','Adadelta'] }

grid_search  = GridSearchCV(estimator = grid_model,
                            param_grid = parameters,
                            scoring = 'accuracy',
                            cv = 2)


In [ ]:
grid_search = grid_search.fit(X_train,y_train)

In [ ]:

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
# Let us check our best parameters
best_parameters

In [ ]:
# Let us check our best accuracy got through grid search
best_accuracy

In [ ]:
plt.plot(lst_pred.history['loss'], label= 'loss(mse)')
plt.plot(np.sqrt(lst_pred.history['mean_squared_error']), label= 'rmse')
plt.legend(loc=1)

In [ ]:
from sklearn.metrics import mean_squared_error
import math

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train, y_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

In [ ]:
# creating submission file 
submission_pfs = my_model.predict(X_test)
# we will keep every value between 0 and 20
submission_pfs = submission_pfs.clip(0,20)
# creating dataframe with required columns 
submission = pd.DataFrame({'ID':test['ID'],'item_cnt_month':submission_pfs.ravel()})
# creating csv file from dataframe


In [ ]:
submission.to_csv('sub_pfs.csv',index = False)